# An attempt at using the cogent API
An attempt made by an electromechanical engineer, not a programmer, so be warned

## First aid with Linked Data Event Stream
The collection is semantically stored: there is no fixed data model necessary. It is possible to access the data from different angels.

Depending on the time you have & your interest, I curated some articles on this approach:

* If you know nothing about LDES or semantic descriptions, following article explains it in 8 minutes https://medium.com/@samuel.vanackere/linked-data-event-streams-explained-in-8-minutes-e1c76d077bb9 
* If you want to know how to backend of the collection of Ghent functions, you can view following 6 minutes video: https://youtu.be/8_0QSEj_Qhg
* If you want to have the broader picture (why does LDES exists, where does it fit in within the internet landscape, who is building it...), check out following course (+-2 hours): https://academy.europa.eu/course/view.php?id=239 





## LDES event stream of the collection of Ghent

The base URL of the collection is https://apidg.gent.be/opendata/adlib2eventstream/v1/

I will use python to demonstrate the use.

In [ ]:
import httpx 
import json

In [ ]:
base = "https://apidg.gent.be/opendata/adlib2eventstream/v1/"
datasets = httpx.get(base).json()

print(json.dumps(datasets,indent=4))

In [ ]:
# Let's pick out the first dataset of the catalog:
dataset = datasets["Datasetcatalogus.heeftDataset"][5]

print(dataset["Dataset.beschrijving"])
print(dataset["tree:view"])

In [ ]:
objects = httpx.get(dataset["tree:view"],follow_redirects=True).json()
objects